# Tracing Basics

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [2]:
# You can set them inline
import os
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

In [4]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Tracing with @traceable

The @traceable decorator is a simple way to log traces from the LangSmith Python SDK. Simply decorate any function with @traceable.

The decorator works by creating a run tree for you each time the function is called and inserting it within the current trace. The function inputs, name, and other information is then streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are patched to LangSmith so you can detect and diagnose sources of errors. This is all done on a background thread to avoid blocking your app's execution.

In [5]:
# Import traceable as per TODO
from langsmith import traceable
import requests
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "ollama"
MODEL_NAME = "llama3.2:latest"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

OLLAMA_API_URL = "http://localhost:11434/api/generate"
nest_asyncio.apply()
retriever = get_vector_db_retriever()

# Set up tracing for each function as per TODO
@traceable
def retrieve_documents(question: str):
    return retriever.invoke(question)  # NOTE: This is a LangChain vector db retriever, so this .invoke() call will be traced automatically

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs}\n\nQuestion: {question}"
        }
    ]
    prompt = "\n".join([f"{msg['role']}: {msg['content']}" for msg in messages])
    return call_ollama(prompt)

@traceable
def call_ollama(
    messages: str,  # 'messages' est une str ici, pas une List[dict]
    model: str = MODEL_NAME,
    temperature: float = 0.0
) -> str:
    payload = {
        "model": model,
        "prompt": messages,
        "temperature": temperature,
        "stream": False
    }
    response = requests.post(OLLAMA_API_URL, json=payload)
    response.raise_for_status()
    return response.json()["response"]

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response

USER_AGENT environment variable not set, consider setting it to identify your requests.


@traceable handles the RunTree lifecycle for you!

In [6]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

To use the @traceable decorator, decorate any function with @traceable. For example:

from langsmith import traceable
@traceable
def my_function():
    pass

Then, to toggle tracing on or off without changing your code, set the LANGSMITH_TRACING environment variable to 'true'.


##### Let's take a look in LangSmith!

### Adding Metadata

LangSmith supports sending arbitrary metadata along with traces.

Metadata is a collection of key-value pairs that can be attached to runs. Metadata can be used to store additional information about a run, such as the version of the application that generated the run, the environment in which the run was generated, or any other information that you want to associate with a run. Similar to tags, you can use metadata to filter runs in the LangSmith UI, and can be used to group runs together for analysis.

In [12]:
from langsmith import traceable

@traceable(
    metadata={"vectordb": "sklearn"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs}\n\nQuestion: {question}"
        }
    ]
    prompt = "\n".join([f"{msg['role']}: {msg['content']}" for msg in messages])
    return call_ollama(prompt)

@traceable(
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_ollama(
    messages: str,  # 'messages' est une str ici, pas une List[dict]
    model: str = MODEL_NAME,
    temperature: float = 0.0
) -> str:
    payload = {
        "model": model,
        "prompt": messages,
        "temperature": temperature,
        "stream": False
    }
    response = requests.post(OLLAMA_API_URL, json=payload)
    response.raise_for_status()
    return response.json()["response"]

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response


In [13]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

To add metadata to a run using the `@traceable` decorator, you can pass a dictionary of key-value pairs to the `meta` parameter. For example:

```python
from langsmith import traceable

@traceable(meta={"version": "1.0", "env": "dev"})
def my_run():
    # code here
```

This will add metadata to the run with the specified key-value pairs.


You can also add metadata at runtime!

In [14]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

You can set metadata at runtime by passing the `langsmith_extra` parameter when calling the function, or by using the context manager with the `metadata` and `tags` parameters. Additionally, you can dynamically set metadata on the parent run by accessing its `metadata` attribute before running the code.


##### Let's take a look in LangSmith!